## doc2vec training excercise

In this excercise, you will train a Paragraph Vectors / doc2vec model using gensim. You can find information on the gensim doc2vec api here: https://radimrehurek.com/gensim/models/doc2vec.html

N.B. You should be using Python 3 for this.

The data folder contains a train and test set with small sets of documents from the "20 newsgroups" dataset.

What we're going to do is the following:
* Read a dataset with documents
* Transform each document into a list of tokens (words)
* Train a doc2vec model (DM)
* Train a second model (DBOW)
* Inspect the outcomes a bit

In [1]:
import os
from gensim.models import doc2vec
from gensim.utils import simple_preprocess

In [2]:
# generic settings
HOMEDIR = './'
CORPUS_FILE = os.path.join(HOMEDIR, "data/train_docs.txt")

# file names for the models we'll be creating
MODEL_FILE_DM = os.path.join(HOMEDIR, "models/doc2vec_DM_v20171229.bin")
MODEL_FILE_DBOW = os.path.join(HOMEDIR, "models/doc2vec_DBOW_v20171229.bin")

**Read the corpus. Each line is a document / paragraph. Optionally preprocess it first.**

In [3]:
flg_preprocess = False

if flg_preprocess:
    # quick & simple approach
    docs = doc2vec.TaggedLineDocument(CORPUS_FILE)
else:
    # with pre-processing
    with open(CORPUS_FILE, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        docs = [simple_preprocess(line, deacc=False, min_len=1) for line in lines]
        docs = [doc2vec.TaggedDocument(doc, tags=[i]) for i, doc in enumerate(docs)]

In [5]:
# have a look at the data
docs[0]

TaggedDocument(words=['anarchism', 'is', 'a', 'political', 'philosophy', 'that', 'advocates', 'self', 'governed', 'societies', 'with', 'voluntary', 'institutions', 'these', 'are', 'often', 'described', 'as', 'stateless', 'societies', 'but', 'several', 'authors', 'have', 'defined', 'them', 'more', 'specifically', 'as', 'institutions', 'based', 'on', 'non', 'hierarchical', 'free', 'associations', 'anarchism', 'holds', 'the', 'state', 'to', 'be', 'undesirable', 'unnecessary', 'or', 'harmful', 'while', 'anti', 'statism', 'is', 'central', 'anarchism', 'entails', 'opposing', 'authority', 'or', 'hierarchical', 'organisation', 'in', 'the', 'conduct', 'of', 'human', 'relations', 'including', 'but', 'not', 'limited', 'to', 'the', 'state', 'system'], tags=[0])

## Training a DM (Distributed Memory) model

In [6]:
# train DM model
model_dm = doc2vec.Doc2Vec(docs, 
                           size=200, # vector size, should be the same size as pre-trained embedding size when not using dm_concat
                           window=10, # window size for word context, on each side
                           min_count=1, # minimum nr. of occurrences of a word
                           sample=1e-5, # threshold for undersampling high-frequency words
                           workers=4, # for multicore processing
                           hs=0, # if 1, use hierarchical softmax; if 0, use negative sampling
                           dm=1, # if 1 use PV-DM, if 0 use PM-DBOW
                           negative=5, # how many words to use for negative sampling
                           dbow_words=1, # train word vectors
                           dm_concat=1, # concatenate vectors or sum/average them?
                           iter=100 # nr of epochs to train
                          )

In [7]:
# save it for later use
model_dm.save(MODEL_FILE_DM)

## Training a DBOW (Distributed Bag Of Words) model

**_Excercise 1: Train a DBOW model_**

It's very similar to the previous command. What should you change?

In [ ]:
# train DBOW model
...enter your code here...

In [8]:
# solution
model_dbow = doc2vec.Doc2Vec(docs, 
                            size=200, # vector size, should be the same size as pre-trained embedding size when not using dm_concat
                            window=10, # window size for word context, on each side
                            min_count=1, # minimum nr. of occurrences of a word
                            sample=1e-5, # threshold for undersampling high-frequency words
                            workers=4, # for multicore processing
                            hs=0, # if 1, use hierarchical softmax; if 0, use negative sampling
                            dm=0, # if 1 use PV-DM, if 0 use PM-DBOW
                            negative=5, # how many words to use for negative sampling
                            dbow_words=1, # train word vectors
                            iter=100 # nr of epochs to train
                            )

========= END OF EXERCISE ============

In [9]:
# also save this one
model_dbow.save(MODEL_FILE_DBOW)

## **Question: Look at the model files that are now created in the models directory. Can you explain why there are 2 files for the DM model, but only 1 for the DBOW model?**

In [17]:
def show_most_similar(model, docs, ref_doc_id):
    """
    For a given document, display the most similar ones in the corpus
    """
    def print_doc(doc_id):
        doc_txt = ' '.join(docs[doc_id].words)
        print("[Doc {}]: {}".format(doc_id, doc_txt))
        
    print("[Original document]")
    print_doc(ref_doc_id)
    print("\n[Most similar documents]")
    for doc_id, similarity in model.docvecs.most_similar(ref_doc_id, topn=3):
        print("-----------------")
        print("similarity: {}".format(similarity))
        print_doc(doc_id)


In [18]:
show_most_similar(model_dbow, list(docs), 200)

[Original document]
[Doc 200]: single scattering albedo is used to define scattering of electromagnetic waves on small particles it depends on properties of the material lrb refractive index rrb the size of the particle or particles and the wavelength of the incoming radiation

[Most similar documents]
-----------------
similarity: 0.8602718114852905
[Doc 154]: terrestrial albedo
-----------------
similarity: 0.8536304235458374
[Doc 187]: water reflects light very differently from typical terrestrial materials the reflectivity of a water surface is calculated using the fresnel equations lrb see graph rrb
-----------------
similarity: 0.8468575477600098
[Doc 148]: albedo lrb rrb or reflection coefficient derived from latin albedo whiteness lrb or reflected sunlight rrb in turn from albus white is the diffuse reflectivity or reflecting power of a surface


## Prediction phase

In [19]:
test_data_file = os.path.join(HOMEDIR, "data/test_docs.txt")

In [21]:
# read test data: each line into a list of tokens
with open(test_data_file, "r") as f:
    test_docs = [ x.strip().split() for x in f.readlines() ]

In [22]:
# inference hyper-parameters
start_alpha=0.01
infer_epoch=1000

Create the embeddings for the test documents. Remember: this is an inference step that actually trains a network.

In [24]:
test_docvecs = [model_dm.infer_vector(d, alpha=start_alpha, steps=infer_epoch) for d in test_docs]

In [26]:
# see what one document embedding looks like
test_docvecs[0]

array([ -3.34543269e-03,  -2.02659750e-04,   1.83269521e-03,
        -3.59125179e-03,  -9.58812714e-04,   1.32690510e-03,
         6.83834252e-04,   3.22495471e-03,  -2.63553765e-03,
        -1.47204555e-03,  -4.05679084e-03,   1.71870308e-03,
        -1.93525094e-03,   2.16521672e-03,  -2.87479197e-04,
         3.72954877e-03,  -1.40604109e-03,  -3.12495627e-04,
        -1.19268964e-03,  -4.41035285e-04,  -8.22708884e-04,
        -3.64960084e-04,  -7.37239316e-04,  -1.07299641e-03,
        -2.10524863e-03,   3.84913268e-03,   2.21801433e-03,
        -7.07347237e-04,  -7.95700529e-04,  -3.16595496e-03,
        -1.02289414e-04,   5.83516376e-04,  -2.63492903e-03,
        -1.08312233e-03,  -1.02029741e-03,   3.78575549e-03,
         1.45962683e-03,   2.54141851e-05,   1.38039165e-03,
        -1.80106028e-03,  -1.61020248e-03,   6.55168667e-04,
         3.43498797e-03,   1.07691984e-03,  -5.28099656e-04,
         3.18728294e-03,  -1.71532412e-03,  -1.34388974e-03,
        -1.91890911e-04,

### The excercise continues in the next notebook!